In [1252]:
import os
import pandas as pd

import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.rcParams['timezone'] = 'Europe/Berlin'

import data_process_base
import importlib
importlib.reload(data_process_base)

try:
    import seaborn
    colors = seaborn.color_palette()
except:
    colors = ["b", "g", "r", "c", "y"]

In [1253]:
"""
Data processing for power flow time-step iteration

"""

# create random time series from import data
da = data_process_base.DataAction()
keep_cols = ['DE_KN_residential1_grid_import',
             'DE_KN_residential2_grid_import',
             'cet_cest_timestamp']
da.imp_procc("house_data.csv", keep_cols)
night_merge = da.power_merge()

Number of data frame segments =  90
dfList created successfully.


In [1254]:
test = night_merge.iloc[:12,:].copy()
da.sgen_comm(ts=test, wind_length=3, sgen_val=99999, parties=4)

TypeError: choice() takes 2 positional arguments but 3 were given

In [ ]:
import random

all_sgens = ['sgen_1','sgen_2','sgen_3','sgen_4']
sel = random.sample(all_sgens, 3)
sel

['sgen_1', 'sgen_3', 'sgen_4']